<a href="https://colab.research.google.com/github/lexxai/goit_python_ds_hw_05/blob/main/goit_ds_hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## LOAD DATASET

In [1]:
import pandas as pd
import numpy as np

In [2]:
from pathlib import Path

SAVED_DATAFRAME = Path("/content/data.feather")
URL = "https://drive.usercontent.google.com/download?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO&export=download&authuser=0"
OUTPUT = Path("/content/homework.zip")
CSV_DATA_PATH  = Path("/content/data")

In [3]:
if SAVED_DATAFRAME.is_file():
  df = pd.read_feather(SAVED_DATAFRAME)
else:
  df = pd.DataFrame()

In [4]:
if not OUTPUT.is_file() and df.empty:
  !wget -O $OUTPUT $URL

if OUTPUT.is_file() and df.empty and not CSV_DATA_PATH.is_dir():
  !unzip -q -o $OUTPUT
  #!rm $OUTPUT

--2024-01-30 04:04:48--  https://drive.usercontent.google.com/download?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 108.177.127.132, 2a00:1450:4013:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|108.177.127.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4211746 (4.0M) [application/octet-stream]
Saving to: ‘/content/homework.zip’

/content/homework.z 100%[===================>]   4.02M  --.-KB/s    in 0.02s   

2024-01-30 04:05:18 (230 MB/s) - ‘/content/homework.zip’ saved [4211746/4211746]



In [5]:
if df.empty:
  class_path = list(CSV_DATA_PATH.iterdir())
  class_list = list(d.name for d in class_path)
  print(class_list)

['stairs', 'idle', 'walking', 'running']


## BUILD DataFrame

In [6]:
def flatten_frame(frame):
        columns=[f"{col}_{i}" for i in range(frame.shape[0]) for col in frame.columns]
        return pd.DataFrame(frame.values.reshape(1, -1), columns=columns)

In [7]:
def add_stat_feature_frame(frame, rows):
  features = []
  features.append(pd.DataFrame([frame.iloc[:,0].mean()]*rows,columns=['X_mean']))
  features.append(pd.DataFrame([frame.iloc[:,1].mean()]*rows,columns=['Y_mean']))
  features.append(pd.DataFrame([frame.iloc[:,1].mean()]*rows,columns=['Z_mean']))
  result = pd.concat(features, axis=1)
  return result

In [8]:
# df = pd.DataFrame()
if df.empty:
  FLATTEN = True
  STAT_FEATURE = True
  dfws = []
  limit_frames = None
  for class_id, work_class_path in enumerate(class_path):
    list_files = list(sorted(work_class_path.glob('*.csv'), key=lambda path: int(path.stem.rsplit("-", 1)[1])))
    print(f"Importing class '{work_class_path.name:7}' : {class_id}. Frames: {len(list_files)}")
    for i, filename in enumerate(list_files):
      # READ FRAME CSV FILE
      df_w: pd.DataFrame = pd.read_csv(filename)
      addon_features = [df_w]
      if FLATTEN:
        addon_features = [flatten_frame(df_w)]
      if STAT_FEATURE:
        addon_features.append(add_stat_feature_frame(df_w,addon_features[-1].shape[0]))
      df_w = pd.concat(addon_features, axis=1)
      df_w['class'] = class_id
      dfws.append(df_w)
      if limit_frames and (i > limit_frames):
        break
  df = pd.concat(dfws, axis=0, ignore_index=True)
# print(df.info())
print(df.shape)

Importing class 'stairs ' : 0. Frames: 165
Importing class 'idle   ' : 1. Frames: 1039
Importing class 'walking' : 2. Frames: 1850
Importing class 'running' : 3. Frames: 3408
(6462, 94)


In [9]:
df.sample(4)

,accelerometer_X_0,accelerometer_Y_0,accelerometer_Z_0,accelerometer_X_1,accelerometer_Y_1,accelerometer_Z_1,accelerometer_X_2,accelerometer_Y_2,accelerometer_Z_2,accelerometer_X_3,...,accelerometer_X_28,accelerometer_Y_28,accelerometer_Z_28,accelerometer_X_29,accelerometer_Y_29,accelerometer_Z_29,X_mean,Y_mean,Z_mean,class
275,-0.100556,0.225055,9.768343,-0.100556,0.225055,9.753978,-0.095768,0.225055,9.787497,-0.110133,...,-0.316035,0.201113,9.576807,-0.770933,-0.641646,10.065224,-0.129766,0.195686,0.195686,1
1727,-1.982399,-7.062895,-0.689530,-0.229843,-1.288081,5.080496,-1.134852,-7.776367,-0.134075,1.134852,...,7.383718,10.879252,19.177555,-2.341529,-12.061988,-0.282516,-3.282929,-9.063171,-9.063171,2
3304,7.651869,20.422540,0.023942,2.389413,17.808075,-2.011129,-5.525817,-19.756952,-23.305159,4.189853,...,18.966866,35.142094,14.920665,-6.512228,9.686940,-2.327164,4.809472,9.443530,9.443530,3
4936,0.656011,10.769119,-1.551443,4.975151,-7.010222,-7.987056,-0.991199,4.668694,4.544195,-4.548983,...,-4.745308,6.023812,-6.095637,9.050082,-14.575900,3.217807,4.252102,7.535351,7.535351,3


In [10]:
if not df.empty and not SAVED_DATAFRAME.is_file():
  df.to_feather("data.feather")

In [11]:
for x in df['class'].unique():
  count = df['class'][df['class'] == x].count()
  prop = count / df.shape[0]
  print(f"class: {x}, rows: {count:7}, {class_list[x]:7}, prop: {prop:.4}" )

class: 0, rows:     165, stairs , prop: 0.02553
class: 1, rows:    1039, idle   , prop: 0.1608
class: 2, rows:    1850, walking, prop: 0.2863
class: 3, rows:    3408, running, prop: 0.5274


In [12]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
# X = np.array(df.iloc[:,:-1])
# y = np.array(df.iloc[:,-1])

In [13]:
X.shape

(6462, 93)

In [14]:
# X.corr()

### X SCALER

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


### train_test_split X_scaled

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, accuracy_score

SEED = 42

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=SEED, stratify=y)

## SVC

In [17]:
clf = SVC(probability=True)
clf.fit(X_train, y_train)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9824651882413615


### Cross Validation

In [18]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
scores.mean()

0.9827550841000996

### Count classes

In [19]:
y_test_pred = clf.decision_function(X_test)
y_test_pred.round(2)

array([[ 2.19, -0.27,  3.27,  0.8 ],
       [ 0.74, -0.22,  2.22,  3.26],
       [ 0.74, -0.24,  2.21,  3.27],
       ...,
       [ 0.75, -0.26,  2.23,  3.27],
       [ 0.74, -0.26,  2.23,  3.27],
       [ 0.74, -0.24,  2.2 ,  3.27]])

In [20]:
y_test_pred = clf.predict_proba(X_test)
y_test_pred.round(2)

array([[0.09, 0.  , 0.91, 0.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       ...,
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ],
       [0.  , 0.  , 0.  , 1.  ]])

In [21]:
y_test_pred = np.argmax(y_test_pred, axis=1)
y_test_pred

array([2, 3, 3, ..., 3, 3, 3])

In [22]:
y_test = clf.predict(X_test)

In [23]:
print(f"{y_test.shape=}")
y_pred_result = {}
for i in y_test_pred:
  y_pred_result[class_list[i]] = y_pred_result.get(class_list[i],0) + 1
print(f"{y_pred_result=}")

y_test_result = {}
for i in y_test:
  y_test_result[class_list[i]] = y_test_result.get(class_list[i],0) + 1
print(f"{y_test_result=}")

y_test.shape=(1939,)
y_pred_result={'walking': 571, 'running': 1023, 'idle': 312, 'stairs': 33}
y_test_result={'walking': 583, 'running': 1023, 'idle': 312, 'stairs': 21}


### PIPELINE SVC

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)

In [25]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovo'))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9876224858174316


In [26]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovr', kernel='linear'))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9819494584837545


In [27]:
clf = make_pipeline(StandardScaler(), SVC(decision_function_shape='ovr', kernel='poly', degree=4))
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=0.9742135121196493


In [28]:
numeric_features = X.columns
len(numeric_features)


93

## RandomForestClassifier

In [29]:
from sklearn.ensemble import RandomForestClassifier

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(X, y)
print(f"{clf.classes_=}")
print(f"{clf.score(X_test, y_test)=}")

clf.classes_=array([0, 1, 2, 3])
clf.score(X_test, y_test)=1.0


### Cross Validation

In [30]:
scores = cross_val_score(clf, X_train, y_train, cv=10)
scores.mean()

0.9765643009240266

## PIPE TEST

In [31]:
preprocessor = ColumnTransformer(
    transformers=[
        ('scaler', StandardScaler(), numeric_features)
    ])

In [32]:
models = {
    'SVC': {
        'model': SVC(),
        'params': {}
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'params': {}
    }
}

results = {}

In [33]:
# for name, model_info in models.items():
#     pipeline = Pipeline([
#         ('preprocessor', preprocessor),
#         ('classifier', model_info['model'])
#     ])

#     grid_search = GridSearchCV(pipeline,
#                                model_info['params'],
#                                cv=5,
#                                scoring='recall',
#                                )

#     grid_search.fit(X_train, y_train)

# y_pred = grid_search.predict(X_test)

# classification_rep = classification_report(y_test, y_pred)
# confusion_mat = confusion_matrix(y_test, y_pred)

# results[name] = {
#     'model': grid_search,
#     'classification_report': classification_rep,
#     'confusion_matrix': confusion_mat
# }